# 승자의 지혜 - 8등 소스코드 분석 2

### 3. 머신러닝 모델

#### - train_predict()에서는 데이터를 훈련/검증/테스트 3가지로 구분 
#### - 첫 번째 train_predict
#### 1.훈련데이터의 가장 최신 날짜인 '2016-05-28'을 테스트 데이터와 같이 사용, 나머지인 '2015-01-28'~'2016-05-27' 기간은 훈련 데이터와 검증 데이터 8:2 비율로 분리 
#### 2. 검증 데이터로 최적 파라미터 결정
#### 3. 모든 훈련 데이터를 머신러닝 모델에 학습시키고 (xgb, lgbm, 앙상블)
#### 4. 테스트 데이터에 대한 예측값 구함 
#### 5. 평가 척도인 MAP@7값을 확인하여, 모델의 성능을 검증 
#### - 두 번째 train_predict: 실제 훈련 데이터를 모두 사용, '2016-06-28' 날짜에 대한 예측 

#### 1명의 고객이 같은 날짜에 신규 상품을 2개 이상 구매하는 경우, 해당 고객의 구매내역은 2줄을 차지하게 된다. 이럴 경우, '신규 구매' 건수가 많은 고객, 구매활동량이 많은 고객에 대한 분포가 필요 이상으로 많아져, 평범한 고객들에 대한 예측 정확도가 낮아질 위험이 있다. 따라서 신규 구매 건수가 많은 고객에 대해 weight을 낮게 배정하여 올바른 데이터 분포를 유지한다. 

In [ ]:
train_predict(all_df, features, prod_features, "2016-05-28", cv=True)
train_predict(all_df, features, prod_features, "2016-06-28", cv=False)

In [30]:
def train_predict(all_df, features, prod_features, str_date, cv):
    # all_df : 통합 데이터
    # features : 학습에 사용할 변수
    # prod_features : 24개 금융 변수
    # str_date : 예측 결과물을 산출하는 날짜. 2016-05-28일 경우, 훈련 데이터의 일부이며 정답을 알고 있기에 교차 검증을 의미하고, 2016-06-28일 경우, 캐글에 업로드하기 위한 테스트 데이터 예측 결과물을 생성한다
    # cv : 교차 검증 실행 여부

    # str_date로 예측 결과물을 산출하는 날짜를 지정한다
    test_date = date_to_int(str_date)
    # 훈련 데이터는 test_date 이전의 모든 데이터를 사용한다
    train_df = all_df[all_df.int_date < test_date]
    # 테스트 데이터를 통합 데이터에서 분리한다
    test_df = pd.DataFrame(all_df[all_df.int_date == test_date])

    # 신규 구매 고객만을 훈련 데이터로 추출한다
    X = []
    Y = []
    for i, prod in enumerate(products):
        prev = prod + "_prev1"
        # 신규 구매 고객을 prX에 저장한다
        prX = train_df[(train_df[prod] == 1) & (train_df[prev] == 0)]  # 0에서 1로 바뀌는 그 순간을 저장ㄴㄱ 
        # prY에는 신규 구매에 대한 label 값을 저장한다 
        prY = np.zeros(prX.shape[0], dtype=np.int8) + i  # np.zeros: 0으로 채워진 shape 차원의 ndarray 반환 
        X.append(prX)
        Y.append(prY)

    XY = pd.concat(X)
    Y = np.hstack(Y)  # hstack: 두 배열을 왼쪽에서 오른쪽으로 붙이기 
    # XY는 신규 구매 데이터만 포함한다
    XY["y"] = Y

    # 메모리에서 변수 삭제
    del train_df
    del all_df

    # 데이터별 가중치를 계산하기 위해서 새로운 변수 (ncodpers + fecha_dato)를 생성한다
    XY["ncodepers_fecha_dato"] = XY["ncodpers"].astype(str) + XY["fecha_dato"]  # 고객식별번호와 날짜를 합친다 
    uniqs, counts = np.unique(XY["ncodepers_fecha_dato"], return_counts=True)  # return_counts=True로 설정하면, 각각의 unique한 아이템이 몇 회 나오는지 알려줌 
    # 자연 상수(e)를 통해서, count가 높은 데이터에 낮은 가중치를 준다
    weights = np.exp(1/counts - 1)  # counts 값이 1이면, e^0=1이므로 가중치는 1이다(가중치 없음). 

    # 가중치를 XY 데이터에 추가한다
    wdf = pd.DataFrame()
    wdf["ncodepers_fecha_dato"] = uniqs  # unique한 아이템들을 넣는다 
    wdf["counts"] = counts
    wdf["weight"] = weights
    XY = XY.merge(wdf, on="ncodepers_fecha_dato")

    # 교차 검증을 위하여 XY를 훈련:검증 (8:2)로 분리한다
    mask = np.random.rand(len(XY)) < 0.8  # 랜덤하게 80%를 고른다.
    XY_train = XY[mask]  # 80%는 훈련 데이터로 넣고
    XY_validate = XY[~mask]  # 20%는 검증 데이터로 넣는다 

    # 테스트 데이터에서 가중치는 모두 1이다
    test_df["weight"] = np.ones(len(test_df), dtype=np.int8)  # 모두 1로 채워넣기 (np.zeros 사촌) 

    # 테스트 데이터에서 “신규 구매” 정답값을 추출한다. 
    test_df["y"] = test_df["ncodpers"]  # 고객식별번호를 테스트 데이터셋의 y값으로 넣는다 
    Y_prev = test_df.as_matrix(columns=prod_features) 
    for prod in products:
        prev = prod + "_prev1"
        padd = prod + "_add"
        # 신규 구매 여부를 구한다
        test_df[padd] = test_df[prod] - test_df[prev]

    test_add_mat = test_df.as_matrix(columns=[prod + "_add" for prod in products])
    C = test_df.as_matrix(columns=["ncodpers"])
    test_add_list = [list() for i in range(len(C))]
    # 평가 척도 MAP@7 계산을 위하여, 고객별 신규 구매 정답값을 test_add_list에 기록한다
    count = 0
    for c in range(len(C)):
        for p in range(len(products)):
            if test_add_mat[c,p] > 0:
                test_add_list[c].append(p)
                count += 1
    
    # 교차 검증에서, 테스트 데이터로 분리된 데이터가 얻을 수 있는 최대 MAP@7 값을 계산한다. 
    if cv:
        max_map7 = mapk(test_add_list, test_add_list, 7, 0.0)
        map7coef = float(len(test_add_list)) / float(sum([int(bool(a)) for a in test_add_list])) # 참이면 1, 거짓이면 0 이므로 분모는 참인 경우의 갯수를 세게 된다. 
        print("Max MAP@7", str_date, max_map7, max_map7*map7coef) # 모두 참인 경우의 MAP@7 값을 계산 

    # LightGBM 모델 학습 후, 예측 결과물을 저장한다
    Y_test_lgbm = engines.lightgbm(XY_train, XY_validate, test_df, features, XY_all = XY, restore = (str_date == "2016-06-28"))
    test_add_list_lightgbm = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.lightgbm.csv.gz" % str_date, "wb"), Y_test_lgbm - Y_prev, C) 
                # io.BytesIO: 메모리에 있는 바이트 배열을 파일처럼 다룰 수 있게 해주는 클래스. 유사한 클래스로 io.StringIO(문자열을 텍스트 파일처럼 취급할 수 있게 해줌)가 있다.
                # gzip.open: 압축된 파일 열기 

    # 교차 검증일 경우, LightGBM 모델의 테스트 데이터 MAP@7 평가 척도를 출력한다
    if cv:
        map7lightgbm = mapk(test_add_list, test_add_list_lightgbm, 7, 0.0)
        print("LightGBMlib MAP@7", str_date, map7lightgbm, map7lightgbm*map7coef)

    # XGBoost 모델 학습 후, 예측 결과물을 저장한다
    Y_test_xgb = engines.xgboost(XY_train, XY_validate, test_df, features, XY_all = XY, restore = (str_date == "2016-06-28"))
    test_add_list_xgboost = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.xgboost.csv.gz" % str_date, "wb"), Y_test_xgb - Y_prev, C)

    # 교차 검증일 경우, XGBoost 모델의 테스트 데이터 MAP@7 평가 척도를 출력한다
    if cv:
        map7xgboost = mapk(test_add_list, test_add_list_xgboost, 7, 0.0)
        print("XGBoost MAP@7", str_date, map7xgboost, map7xgboost*map7coef)

    # 곱셈 후, 제곱근을 구하는 방식으로 앙상블을 수행한다
    Y_test = np.sqrt(np.multiply(Y_test_xgb, Y_test_lgbm))
    # 앙상블 결과물을 저장하고, 테스트 데이터에 대한 MAP@7 를 출력한다
    test_add_list_xl = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.xgboost-lightgbm.csv.gz" % str_date, "wb"), Y_test - Y_prev, C)

    # 정답값인 test_add_list와 앙상블 모델의 예측값을 mapk 함수에 넣어, 평가 척도 점수를 확인한다
    if cv:
        map7xl = mapk(test_add_list, test_add_list_xl, 7, 0.0)
        print("XGBoost+LightGBM MAP@7", str_date, map7xl, map7xl*map7coef) 